# Обработка данных с помощью pandas и создание простого интерактивного DashBoard
### ЗАДАНИЕ

задача - подготовить и обработать исходные данных так, чтобы их можно было использовать во второй части задания.
```
    Требования к выходным данным:
    1) В выходной таблице должны остаться только следующие колонки:
area, cluster, cluster_name, keyword, x, y, count, color, где
●	 area - область,
●	 cluster - номер кластера,
●	 cluster_name - название кластера,
●	 keyword - словосочетание,
●	 count - показатель,
●	 x и y - координаты для диаграммы рассеяния,
●	 color - цвет точки на карте для данного словосочетания

    2) Колонку color нужно добавить самостоятельно - цвета вы можете взять из цветовых палеток Tableu или по своему усмотрению.
    3) Цвет задается каждому словосочетанию согласно следующими правилам:
●	внутри одной области цвета словосочетаний в одном кластере должны быть одинаковые, в разных - отличаться (например, у "Кластер 1" все слова будут окрашены в красный, у "Кластер 2" - в зеленый и т.д.)
●	цвета кластеров в разных областях могут повторяться
●	цвета кластеров в разных областях с разным номером не имеют никакой связи (у одной области [area] слова из "Кластер 1" могут быть красного цвета, в другой области у слов из "Кластер 1" может быть другой цвет)
    4) Не должно быть дубликатов слов в одной и той же области (area), но словосочетание может повторяться из area в area
    5) Колонки должны называться именно так, как указано в п.1
    6) Сортировка должна происходить по колонкам area, cluster, cluster_name, count (по count значения сортируются в убывающем порядке, в остальных - по возрастающему).
    7) Количество переданных в исходных ключевых слов должно совпадать с количество слов в выходных данных (за исключением дублированных строк или строк с пустыми\неформатными значениями по ключевым показателям [перечислены в п. 1], если такие имеются).
    8) Никакие другие особенности оформления не должны учитываться при обработке данных (заливка и пр.)
    9) Выходные данные должны быть аккуратно оформлены (заголовки закреплены, включен фильтр)
```

In [1]:
# пакеты для обработки данных
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
# пакеты для визуализации
!pip install dash
import dash
from dash import dcc
from dash import html
import pandas as pd
import plotly.graph_objs as go



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Посмотрим, что за DataFrame

In [3]:
#Установим настройки вывода 
pd.options.display.width = None
pd.options.display.max_columns = None
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', 10)
pd.set_option('display.width', 1000)

In [4]:
df = pd.read_csv('tz_data.csv') 
df

,area,cluster,cluster_name,keyword,good (1),count,x,y
0,eligibility,0.0,Кластер 0,several animated buried,1.0,1260,5.772342,12.564796257345005
1,eligibility,0.0,Кластер 0,singles unusual buyers,1.0,866,14.829280,7.8507285727125815
2,eligibility,0.0,Кластер 0,hawaiian directive,1.0,163,11.381856,3.8981370219558604
3,eligibility,0.0,Кластер 0,dynamics directly,1.0,1146,9.980149,6.281427914064545
4,eligibility,1.0,Кластер 1,decision surgeons montreal,1.0,823,3.283940,4.39674063521296
...,...,...,...,...,...,...,...,...
224,greetings,2.0,Кластер 2,disposition layout,1.0,279,10.971214,4.857810387061303
225,greetings,2.0,Кластер 2,sapphire grounds,1.0,335,1.160626,3.642819729434763
226,greetings,3.0,Кластер 3,entire ethical speakers,1.0,1782,7.985910,6.003699268483375
227,greetings,3.0,Кластер 3,courtesy textiles diameter,1.0,84,0.509490,4.151198803764073


In [5]:
print('Исходная размерность даты:', df.shape)  # посмотрели размерность даты

Исходная размерность даты: (229, 8)


In [6]:
df.pop('good (1)')

0      1.0
1      1.0
2      1.0
3      1.0
4      1.0
      ... 
224    1.0
225    1.0
226    1.0
227    1.0
228    1.0
Name: good (1), Length: 229, dtype: float64

In [7]:
# считаем пропуски
df2 = df.isnull().sum(axis=0)

In [8]:
df2

area            1
cluster         1
cluster_name    1
keyword         1
count           2
x               1
y               1
dtype: int64

In [9]:
df2 = df.dropna()
print(f"После удаления пропусков в дате количество строк уменьшилось на:{df.shape[0] - df2.shape[0]}")

После удаления пропусков в дате количество строк уменьшилось на:2


In [10]:
# посмотрим основные описательные статистики по числовым столбцам и столбцам с объектами (категориальные переменные)

df2.describe(), df2.describe(include='object')

(          cluster           x
 count  227.000000  227.000000
 mean     1.396476    7.682790
 std      1.081455    4.363280
 min      0.000000    0.039448
 25%      0.000000    3.610201
 50%      1.000000    8.042422
 75%      2.000000   11.390670
 max      3.000000   14.927879,
                area cluster_name                  keyword count                   y
 count           227          227                      227   227                 227
 unique           15            4                      193   208                 224
 top     eligibility    Кластер 1  offset cnetcom applying   161  0.7791614555083071
 freq             16           62                        3     3                   2)

**выяснили, что числовые описательные статистики не все по числам, есть проблема в типе данных по столбцам**

In [11]:
df2.dtypes

area             object
cluster         float64
cluster_name     object
keyword          object
count            object
x               float64
y                object
dtype: object

In [12]:
df2[['count', 'y']] = df2.loc[:,['count', 'y']].apply(pd.to_numeric, errors='coerce') #ошибки заменить на NaN

C:\Users\Никита\AppData\Local\Temp\ipykernel_17936\1463735087.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2[['count', 'y']] = df2.loc[:,['count', 'y']].apply(pd.to_numeric, errors='coerce') #ошибки заменить на NaN


In [13]:
df2.dtypes   

area             object
cluster         float64
cluster_name     object
keyword          object
count           float64
x               float64
y               float64
dtype: object

**теперь по типу данных все корректно**

In [14]:
df3 = df2.dropna() #убрали все пропуски


In [15]:
df3.describe(), df3.describe(include ='object') #посмотрим основные описательные  статистики 

(          cluster        count           x           y
 count  224.000000   224.000000  224.000000  224.000000
 mean     1.401786   986.718750    7.628671    7.043125
 std      1.083590   587.951598    4.361012    4.253755
 min      0.000000    19.000000    0.039448    0.060807
 25%      0.000000   470.500000    3.595513    3.605627
 50%      1.000000  1008.000000    8.010815    6.735408
 75%      2.000000  1496.000000   11.363495   10.298580
 max      3.000000  1986.000000   14.927879   14.923944,
                area cluster_name                  keyword
 count           224          224                      224
 unique           15            4                      191
 top     eligibility    Кластер 1  offset cnetcom applying
 freq             16           61                        3)

**посмотрим, насколько изменилась размерность DataFrame по сравнению с исходной таблицей** 

In [16]:
f'Размерность изменилась на: {(100 * (df.shape[0] - df3.shape[0]) / df.shape[0]).__round__(2)} %'

'Размерность изменилась на: 2.18 %'

**проверим, есть ли дубликаты слов в одной и тойже области, но словосочетание может повторятся из области в область**

In [17]:
df3_1 = df3.groupby(['area']).agg({'keyword': ['count']})    #подсчитаем общее количество элементов
df3_2 = df3.groupby(['area']).agg({'keyword': ['nunique']})  #подсчитаем количество уникальных элементов

In [18]:
df4 = pd.concat([df3_1, df3_2], ignore_index=False, axis=1) # склеим две таблички 
df4.columns = ['counts', 'uniques']         # дадим название столбцам 
df4['delta'] = df4['counts'] - df4['uniques'] #подсчитаем разность 

In [19]:
df4, df3.shape

(             counts  uniques  delta
 area                               
 ar\vr            15       14      1
 available        15       15      0
 capability       15       15      0
 dialog           15       14      1
 eligibility      16       14      2
 except           15       15      0
 greetings        15       14      1
 housewives       14       14      0
 lithuania        15       15      0
 locator          15       15      0
 personnel        15       15      0
 protein          14       14      0
 twisted          16       14      2
 winner           15       15      0
 worlds           14       14      0,
 (224, 7))

In [20]:
#код удалит дубликаты только в тех строках, где значения в колонках "область" и "слово" совпадают
df5 = df3.drop_duplicates(subset=['area', 'keyword'], keep='last') 

In [21]:
df5.shape

(217, 7)

```
присвоим цвет каждому из кластеров. Так как цвета кластеров в разных областях могут повторяться, а сами цвета
у разных кластеров должны отличаться, то достаточно присвоить свой цвет каждому из кластеров в DataFrame
```

In [22]:
#посмотрим, сколько всего кластеров:
df5.groupby('cluster').agg({'keyword': ['count']})


,keyword
,count
cluster,
0.0,56
1.0,58
2.0,58
3.0,45


In [23]:
conditions = [
    (df5['cluster'] == 0),
    (df5['cluster'] == 1),
    (df5['cluster'] == 2),
    (df5['cluster'] == 3)
]                                 #список условий

values = ['red', 'yellow', 'green', 'blue']   # список цветов
df5['colors'] = np.select(conditions, values) #создаем новый столбец с цветами 




C:\Users\Никита\AppData\Local\Temp\ipykernel_17936\1154198487.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df5['colors'] = np.select(conditions, values) #создаем новый столбец с цветами


In [24]:
df5


,area,cluster,cluster_name,keyword,count,x,y,colors
0,eligibility,0.0,Кластер 0,several animated buried,1260.0,5.772342,12.564796,red
1,eligibility,0.0,Кластер 0,singles unusual buyers,866.0,14.829280,7.850729,red
3,eligibility,0.0,Кластер 0,dynamics directly,1146.0,9.980149,6.281428,red
4,eligibility,1.0,Кластер 1,decision surgeons montreal,823.0,3.283940,4.396741,yellow
5,eligibility,1.0,Кластер 1,knives everybody,1377.0,5.607192,13.155189,yellow
...,...,...,...,...,...,...,...,...
224,greetings,2.0,Кластер 2,disposition layout,279.0,10.971214,4.857810,green
225,greetings,2.0,Кластер 2,sapphire grounds,335.0,1.160626,3.642820,green
226,greetings,3.0,Кластер 3,entire ethical speakers,1782.0,7.985910,6.003699,blue
227,greetings,3.0,Кластер 3,courtesy textiles diameter,84.0,0.509490,4.151199,blue


**выгрузим обработанный DataFrame в xlsx файл**

In [25]:
df5.to_excel("output2.xlsx")

# Cоздание интерактивного дашборда

In [26]:

app = dash.Dash(name = "HSE_test_dashbord")  #дали название

In [27]:
# создаем список опций для фильтрации по областям
region_options = [{'label': area, 'value': area} for area in df5['area'].unique()]

In [28]:
# создаем график для начальной области
fig = go.Figure()
region_data = df5[df5['area'] == 'eligibility'] #присвоим начальное значение из области, равное eligibility 

In [29]:
#нанесем точки на график 
for keyword_counter in region_data['keyword']:
    fig.add_trace(go.Scatter(x=region_data[region_data['keyword'] == keyword_counter]['x'],
                             y=region_data[region_data['keyword'] == keyword_counter]['y']
                             )
                  )

In [30]:
# создаем функцию для обновления данных при изменении фильтра, чтобы она активировалась при изменении выбранного фильтра area используем колбэки 

@app.callback(
    dash.dependencies.Output('graph', 'figure'),            #выход будет объект с именем 'graph' со значением 'figure' 
    [dash.dependencies.Input('region-filter', 'value')]     #на вход передается объект с именем 'region-filter' со значением 'value'
    )

def update_figure(selected_region):

    if selected_region is None:
        filtered_data = df5
    else:
        #создаем DataFrame, который будет содержать только значение по выбранному фильтру, т.е. по area
        filtered_data = df5[df5['area'] == selected_region].sort_values(by='y', ascending=False).reset_index(drop=True)
        #сортируем по y, чтобы потом реализовать алгоритм минимизации наложения
    fig = go.Figure()   #создаем область для графика 
    
    # Алгоритм переноса строки. Если в keyword более трех слов, разделить словосочетание по второму пробелу
    for keyword_counter in filtered_data['keyword']:   #проходимся по конкретному keyword
        if len(keyword_counter.split()) >= 3:          # если больше трех слов
            words = keyword_counter.split()            # создаем список из слов, которые содержит словосочетание
            substring = ' '.join(words[:2])            # промежуточной строке присваиваем первые два слова
            subsubstring = ' '.join(words[2:])         # подпромежуточной строке присваиваем остальные слова
            sub_keyword_counter = substring + '<br>' + subsubstring #соединяем слова через оператор переноса строки для html
           #меняем keyword в filtredData на keyword с разделителем строки
            filtered_data.loc[filtered_data['keyword'] == keyword_counter, 'keyword'] = sub_keyword_counter  
            keyword_counter = sub_keyword_counter #обновляем счетчик 
        else:
            keyword_counter = keyword_counter     #если менее трех слов оставляем слово
            
    # Алгоритм минимизации наложения. 
    #Если координаты по y близки для точек, то приналожении на график подписи одну подпись сделать вверху, другую внизу. 
        text_position_list = ['bottom center', 'top center', 'middle center']       #создаем список с наименованиями положения текста

        index = filtered_data[filtered_data['keyword'] == keyword_counter].index[0] #берем текущей индекc по от текущего keyword_counter
       
        if index + 2 <= filtered_data.shape[0]:    #проверяем условие, если индекс не выходит за размерность DataFrame, то сравниваем координаты
            if ((abs(filtered_data.loc[index, 'y'] - filtered_data.loc[index + 1, 'y'])/filtered_data.loc[index, 'y']) <= 0.05):
                #если разница менее 5%, то присваиваем одну позицию (в данном случае это будет счетчик i)
                i = 1
            if ((abs(filtered_data.loc[index, 'y'] - filtered_data.loc[index + 1, 'y'])/filtered_data.loc[index, 'y']) > 0.05):
                #если разница более 5%, то присваиваем другую позицию
                i = 0
        elif index + 2 > filtered_data.shape[0]:
            #чтобы не было ошибки, если подошли к последнему индексу присвоить какое-то значение
            i = 2
            
    # Добавляем точечный график
        fig.add_trace(go.Scatter(
            x=filtered_data[filtered_data['keyword'] == keyword_counter]['x'],  
            y=filtered_data[filtered_data['keyword'] == keyword_counter]['y'],
            mode='markers+text',  #отображать точки и текст к точкам на графике
            textposition=text_position_list[i],   #положение текста в соответствии со счетчиком
            marker=dict(                          #параметры точек
                size=filtered_data[filtered_data['keyword'] == keyword_counter]['count'],  # размер точки в соответствии со столбцом 'count'
                sizemin=10,  #минимальный размер, чтобы точки с маленьким 'count' не были очень маленькими
                sizeref=30,  #уменьшаем все точки в 30 раз, чтобы точки с большим 'count' не заполнили весь график
                sizemode='diameter',  
                color=filtered_data[filtered_data['keyword'] == keyword_counter]['colors'], #задаем цвет точки в соответствии со столбцом из DataFrame 
                opacity=0.5,      #наполовину прозрачные
                showscale=False,  #не вносить в легенду
            ),
            #сам текст берем из колонки 'keyword', которая уже изменена алгоритмом переноса для слов
            text=filtered_data.loc[filtered_data['keyword'] == keyword_counter, 'keyword'], 
            showlegend = False  #скрываем легенду самих точек
        )
        )
        fig.update_layout(width=1400, height=700)              #устанавливаем размер графика
        fig.update_layout(#legend_title_text='Размер точек',
                          xaxis_title='Координата точки Х',    #подпись по х
                          yaxis_title='Координата точки Y',    #подпись по y
                          plot_bgcolor='linen'                 #более менее красивый цвет...
                          )

    return fig

In [31]:
#Легенду для кластеров создадим исскуственно, добавив на график просто цветные точки

# создаем интерфейс с фильтром
app.layout = html.Div([
    html.Label('Диаграмма рассеяния для каждой area'),
    dcc.Dropdown(                   #создаем выпадающий список
        id='region-filter',         #наименование
        options=region_options,     #первоначальный график
        value='eligibility'         #значение
    ),
    dcc.Graph(id='graph', figure=fig)   #добавляем график , его наименование будет 'graph', a значение fig
])

if __name__ == '__main__':      #запускаем приложение
    app.run_server(debug=True)